# Runs Reddit API to collect posts containing a search term, and returns post with comments on it

### Import the Reddit credentials and modules required

In [1]:
from redditcredentials import CLIENT_ID, CLIENT_SECRET, USERNAME_REDDIT, PASSWORD_REDDIT
import requests

### Setup the request for Reddit API, OAuth authentication

In [2]:
# Authorisation code from https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c, as tried what API github suggested and did not work
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)

data = {'grant_type': 'password',
        'username': USERNAME_REDDIT,
        'password': PASSWORD_REDDIT}

headers = {'User-Agent': 'TopicSentimentAcrossSN/0.0.1'}

res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

TOKEN = res.json()['access_token']

headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

### Set search term and create the dictionary to hold posts

In [3]:
search_term = 'Scotrail'
reddit_posts = {}

### Function to get the comments from the posts

In [4]:
def getCommentsFromPosts(post_id, post_title):
        comment_results = requests.get("https://oauth.reddit.com/r/all/comments/" + str(post_id),
                                headers=headers, params={'limit': '25'})

        print("PRINTING COMMENT RESULTS FOR THE TOPIC: \n")

        comments = comment_results.json()[1]['data']['children'][:-1]   #slices last value as it is just list of id's
        all_comments = []

        for comment in comments:
                comment_text = comment['data']['body']
                all_comments += [comment_text]
        
        reddit_posts[post_title] = all_comments

### Find results on search term from reddit posts, and collect comments containing that topic

In [5]:
results = requests.get("https://oauth.reddit.com/r/all/search", 
                headers=headers, params={'limit': '25', 'q':{search_term}})

print("PRINTING SEARCH RESULTS FOR THE TOPIC: \n")

posts = results.json()['data']['children']

for post in posts:
        post_title = post['data']['title']
        print(post_title)
        post_id = post['data']['id']
        getCommentsFromPosts(post_id, post_title)

PRINTING SEARCH RESULTS FOR THE TOPIC: 

Scotrail Strike Going Ahead during COP26 Good Luck to Everybody Involved
PRINTING COMMENT RESULTS FOR THE TOPIC: 

Scotrail Strikes reportedly called off - Glasgow Times
PRINTING COMMENT RESULTS FOR THE TOPIC: 

Massive yes vote in ScotRail pay dispute
PRINTING COMMENT RESULTS FOR THE TOPIC: 

Unite confirm they will not be striking during COP26 after agreement reached with ScotRail
PRINTING COMMENT RESULTS FOR THE TOPIC: 

ScotRail deal appears to be done.
PRINTING COMMENT RESULTS FOR THE TOPIC: 

Scotrail Strikes COP26
PRINTING COMMENT RESULTS FOR THE TOPIC: 

ScotRail and Caledonian Sleeper strike confirmed during COP26
PRINTING COMMENT RESULTS FOR THE TOPIC: 

Rail workers announce 24-hour strike action during COP26 in Scotrail pay dispute
PRINTING COMMENT RESULTS FOR THE TOPIC: 

Scotrail strikes and travelling to Murrayfield
PRINTING COMMENT RESULTS FOR THE TOPIC: 

Scotrail criticised by Aberdeen SNP MP Stephen Flynn for providing insuffi

### Dictionary results

In [6]:
print(reddit_posts)

{'Scotrail Strike Going Ahead during COP26 Good Luck to Everybody Involved': ["Inflation is shooting up, wages are stagnant, and employers are still strapped for funds from the fallout of the pandemic. Energy costs are through the roof and business hasn't come back to many sectors. There will be a winter of strikes ahead I feel.", 'I live in Lenzie and work in Partick. With the trains off my commute has now went from 40 mins to 2 hours and my employer refuses to let me work from home even though I probably could.   \nMon the workers but fuck me this is gonna be a shite 2 weeks :(', "They've been given until 5pm on Wednesday otherwise the strikes are going ahead. I'm screwed getting to work but I hope they get the pay they deserve. Pay people what they're worth.", "If Scotrail and the unions don't agree to resolve the strikes before Cop26, I Graeme Day is booted from his position. It's absolutely shocking that the Scottish Government are letting these strikes go ahead during one of the 

## VADER Analysis test

In [7]:
!pip install vaderSentiment

In [19]:
reddit_results = []
for post in reddit_posts.values():
   # print(post)
    reddit_results.extend(post)

print(reddit_results)

["Inflation is shooting up, wages are stagnant, and employers are still strapped for funds from the fallout of the pandemic. Energy costs are through the roof and business hasn't come back to many sectors. There will be a winter of strikes ahead I feel.", 'I live in Lenzie and work in Partick. With the trains off my commute has now went from 40 mins to 2 hours and my employer refuses to let me work from home even though I probably could.   \nMon the workers but fuck me this is gonna be a shite 2 weeks :(', "They've been given until 5pm on Wednesday otherwise the strikes are going ahead. I'm screwed getting to work but I hope they get the pay they deserve. Pay people what they're worth.", "If Scotrail and the unions don't agree to resolve the strikes before Cop26, I Graeme Day is booted from his position. It's absolutely shocking that the Scottish Government are letting these strikes go ahead during one of the biggest events Scotland has ever had. They have had over seven months of Sund

In [20]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
# Example code:

f = open("reddit_scores.txt", "x")

for reddit_post in reddit_results:
  vs = analyzer.polarity_scores(reddit_post)
  compound = vs['compound']
  sentiment_score = compound * 100
  print("The post '" + reddit_post + "' has a sentiment score of: " + (str(sentiment_score)) + "%")
  
  if (compound >= 0.05):
    print("The post was POSITIVE")
    f.write("\n" + reddit_post + ": RESULT = POSITIVE")
  elif (compound <= -0.05):
    print("The post was NEGATIVE")
    f.write("\n" + reddit_post + ": RESULT = NEGATIVE")
  else:
    print("The post was NEUTRAL")
    f.write("\n" + reddit_post + ": RESULT = NEUTRAL")
  print("{:-<65} {}".format(reddit_post, str(vs)))

f.close()

The tweet 'Inflation is shooting up, wages are stagnant, and employers are still strapped for funds from the fallout of the pandemic. Energy costs are through the roof and business hasn't come back to many sectors. There will be a winter of strikes ahead I feel.' has a sentiment score of: -10.27%
The tweet was NEGATIVE
Inflation is shooting up, wages are stagnant, and employers are still strapped for funds from the fallout of the pandemic. Energy costs are through the roof and business hasn't come back to many sectors. There will be a winter of strikes ahead I feel. {'neg': 0.054, 'neu': 0.901, 'pos': 0.045, 'compound': -0.1027}
The tweet 'I live in Lenzie and work in Partick. With the trains off my commute has now went from 40 mins to 2 hours and my employer refuses to let me work from home even though I probably could.   
Mon the workers but fuck me this is gonna be a shite 2 weeks :(' has a sentiment score of: -86.25%
The tweet was NEGATIVE
I live in Lenzie and work in Partick. With